In [2]:
import numpy as np
from bayspec.model.local import *
from bayspec.model.astro import *
from bayspec.util.prior import unif
from bayspec import DataUnit, Data, Infer, Plot

In [3]:
tbabs_ins = AS_TbAbs()
tbabs_ins.expr = 'tbabs'
tbabs_ins.cfg[1].val = 0
tbabs_ins.par[1].frozen_at(0.467)

ztbabs_ins = AS_TbAbs()
ztbabs_ins.expr = 'ztbabs'
ztbabs_ins.cfg[1].val = 0

pl_ins = pl()

wxt_model = tbabs_ins*ztbabs_ins*pl_ins

In [4]:
wxt = DataUnit(
    src='../wxt/spectrum/p0d00-p160d00.src', 
    bkg='../wxt/spectrum/p0d00-p160d00.bkg', 
    rmf='../wxt/ep.rmf', 
    arf='../wxt/ep.arf', 
    notc=[0.5, 4], 
    stat='cstat', 
    grpg={'min_evt': 5})

wxt_data = Data([('WXT', wxt)])

/Users/jyang/Documents/python_works/bayspec/bayspec/data/spectrum.py:192: UserWarning: ../wxt/spectrum/p0d00-p160d00.src -> src error is not specified and will default to Poisson error
  warnings.warn(f'{src_file} -> src error is not specified and will default to Poisson error')
/Users/jyang/Documents/python_works/bayspec/bayspec/data/spectrum.py:321: UserWarning: ../wxt/spectrum/p0d00-p160d00.bkg -> bkg error is not specified and will default to Poisson error
  warnings.warn(f'{bkg_file} -> bkg error is not specified and will default to Poisson error')


In [5]:
wxt_infer = Infer([(wxt_data, wxt_model)])
wxt_infer.par[2].prior = unif(0.0001, 2)
print(wxt_infer)

╒════════╤═════════════════════╤═════════════╤═════════════╤═════════╕
│  cfg#  │     Expression      │  Component  │  Parameter  │  Value  │
╞════════╪═════════════════════╪═════════════╪═════════════╪═════════╡
│   1    │ ((tbabs*ztbabs)*pl) │    tbabs    │  redshift   │    0    │
├────────┼─────────────────────┼─────────────┼─────────────┼─────────┤
│   2    │ ((tbabs*ztbabs)*pl) │   ztbabs    │  redshift   │    0    │
├────────┼─────────────────────┼─────────────┼─────────────┼─────────┤
│   3    │ ((tbabs*ztbabs)*pl) │     pl      │  redshift   │    0    │
╘════════╧═════════════════════╧═════════════╧═════════════╧═════════╛
╒════════╤═════════════════════╤═════════════╤═════════════╤═════════╤═════════════════╕
│  par#  │     Expression      │  Component  │  Parameter  │  Value  │      Prior      │
╞════════╪═════════════════════╪═════════════╪═════════════╪═════════╪═════════════════╡
│   1    │ ((tbabs*ztbabs)*pl) │    tbabs    │     NH      │  0.467  │     frozen      │
├────

In [6]:
wxt_post = wxt_infer.multinest(nlive=400, resume=True, savepath='./wxt_bsp_nh')

 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  400
 dimensionality =    3
 resuming from previous job
 *****************************************************
 Starting MultiNest
Acceptance Rate:                        0.676352
Replacements:                               4376
Total Samples:                              6470
Nested Sampling ln(Z):                -14.683524
Importance Nested Sampling ln(Z):     -13.660188 +/-  0.192936
  analysing data from ./wxt_bsp_nh/1-.txt
 ln(ev)=  -14.386031096664505      +/-  0.13596884842309326     
 Total Likelihood Evaluations:         6470
 Sampling finished. Exiting MultiNest


In [7]:
print(wxt_post)

╒════════╤═════════════════════╤═════════════╤═════════════╤════════╤══════════╤════════╤══════════════════╕
│  par#  │     Expression      │  Component  │  Parameter  │  Mean  │  Median  │  Best  │    1sigma CI     │
╞════════╪═════════════════════╪═════════════╪═════════════╪════════╪══════════╪════════╪══════════════════╡
│   2    │ ((tbabs*ztbabs)*pl) │   ztbabs    │     NH      │ 0.699  │  0.669   │ 0.559  │  [0.378, 1.007]  │
├────────┼─────────────────────┼─────────────┼─────────────┼────────┼──────────┼────────┼──────────────────┤
│   3    │ ((tbabs*ztbabs)*pl) │     pl      │  $\alpha$   │ -2.161 │  -2.147  │ -1.939 │ [-2.693, -1.634] │
├────────┼─────────────────────┼─────────────┼─────────────┼────────┼──────────┼────────┼──────────────────┤
│   4    │ ((tbabs*ztbabs)*pl) │     pl      │   log$A$    │ -4.12  │  -4.086  │ -3.754 │ [-4.978, -3.252] │
╘════════╧═════════════════════╧═════════════╧═════════════╧════════╧══════════╧════════╧══════════════════╛
╒════════╤═════════

In [8]:
fig = Plot.infer_ctsspec(wxt_post, style='CE', show=False)
fig.write_html('./wxt_bsp_nh/ctsspec.html')

In [9]:
fig = Plot.post_corner(wxt_post, show=False)
fig.write_html('./wxt_bsp_nh/corner.html')